In [2]:
# imports
import pandas as pd

In [3]:
# create column names
features = []
for i in range (1,434):
    feature_name = f"feature_{i}"
    # print(feature_name)
    features.append(feature_name)
# features.append("label")
len(features)

433

In [4]:
# merge test data and labels
def merge(data, labels):
    dataDf = pd.read_csv(data, header=None, names=features, float_precision='round_trip')
    labelsDf = pd.read_csv(labels, header=None, names=['label'])
    # turn labels into ints
    labelsDf['label'] = labelsDf['label'].astype(int)
    merged = pd.merge(dataDf, labelsDf, left_index=True, right_index=True)
    # print(f"data: {dataDf.shape}, labels: {labelsDf.shape}, merged: {merged.shape}")
    # print(f"{data}: \n{dataDf.iloc[4]}")
    return merged


In [5]:
test_merged = merge("original_data/noExclusion_test_data.csv","original_data/noExclusion_test_label.csv")
train_merged = merge("original_data/noExclusion_train_data.csv","original_data/noExclusion_train_label.csv")    
# print(f"test: {test_merged.shape}, train: {train_merged.shape}")
combined = pd.concat([train_merged,test_merged])
combined['label']
combined.to_csv('combined.csv')
test_merged.to_csv('test_merged.csv')
train_merged.to_csv('train_merged.csv')

In [15]:
import random
random.seed(1)
# to use for sampling 236 NDBE records (iCodeLikeImDrunk, 2012)
indexes = random.sample(range(257), 178)
len(indexes)
# indexes

178

In [16]:
x_train = pd.read_csv('original_data/noExclusion_train_data.csv', header=None)
y_train = pd.read_csv('original_data/noExclusion_train_label.csv', header=None, names=['label'])
combined = pd.merge(x_train, y_train, left_index=True, right_index=True)
combined

,0,1,2,3,4,5,6,7,8,9,...,424,425,426,427,428,429,430,431,432,label
0,1.31130,1.30930,1.22250,1.24780,1.14720,1.22080,1.26760,1.27630,1.29030,1.27570,...,0.068851,0.065492,0.045312,0.067299,0.065363,0.058188,0.085106,0.084531,0.042246,3.0
1,1.87500,1.87170,1.91020,1.86830,1.88780,1.85140,1.85270,1.82920,1.84790,1.88730,...,0.018279,0.023191,0.019388,0.019210,0.015802,0.021584,0.020229,0.014808,0.024260,3.0
2,1.37820,1.36660,1.36520,1.37190,1.35590,1.34280,1.32630,1.32380,1.31260,1.30360,...,0.036983,0.035556,0.037337,0.035494,0.041724,0.050014,0.045198,0.053270,0.043443,2.0
3,2.17140,2.15100,2.09500,2.03070,1.99520,2.01720,2.06750,2.05850,2.01780,2.00700,...,0.088781,0.099672,0.130510,0.161320,0.160180,0.181090,0.210100,0.253040,0.297120,2.0
4,0.73976,0.72715,0.71311,0.69704,0.67125,0.68409,0.68809,0.66799,0.66740,0.67134,...,0.099238,0.084927,0.082649,0.096438,0.104410,0.098250,0.099586,0.092213,0.087415,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,0.93847,0.93017,0.91375,0.91752,0.91045,0.90651,0.89648,0.90520,0.88983,0.87055,...,0.021089,0.018307,0.019763,0.021262,0.021234,0.036394,0.024574,0.013140,0.022349,3.0
568,1.00220,0.99335,0.99117,0.98433,0.97987,0.98777,0.98724,1.00410,1.00310,1.00610,...,0.048174,0.051211,0.064251,0.067851,0.072352,0.085265,0.086922,0.102180,0.105880,2.0
569,1.73520,1.73090,1.73770,1.71400,1.68840,1.71110,1.67500,1.66490,1.65220,1.61350,...,0.079610,0.086258,0.089462,0.095490,0.092196,0.119090,0.121690,0.124570,0.134170,2.0
570,1.82100,1.80350,1.89180,1.89530,1.86330,1.85440,1.85490,1.82200,1.86650,1.83400,...,0.081323,0.087510,0.077155,0.070630,0.066664,0.091194,0.096377,0.097625,0.082838,3.0


In [17]:
# make balanced dataset (balance NDBE and neoplasia) via undersampling
# combined = pd.read_csv('combined.csv', index_col=0).reset_index(drop=True)
combined['label'].value_counts()
# 1: 137, 2: 257, 3: 178
squamous = combined.loc[combined['label']==1].reset_index(drop=True)
ndbe = combined.loc[combined['label']==2].reset_index(drop=True)
neoplasia = combined.loc[combined['label']==3].reset_index(drop=True)

# choose the 178 random records
balanced_ndbe = ndbe.iloc[indexes]
# (LondonRob, 2013)
excluded = ndbe.iloc[~ndbe.index.isin(indexes)]
squamous['label'].value_counts()

# merge back together
balanced = pd.concat([squamous, balanced_ndbe, neoplasia]).reset_index(drop=True)
balanced['label'].value_counts()

2.0    178
3.0    178
1.0    137
Name: label, dtype: int64

In [22]:
# make balanced dataset v2
x_train = balanced.drop('label', axis=1)
y_train = balanced['label'].astype(int)

x_train.to_csv('balancedv2/train_data.csv', index=False, header=False)
y_train.to_csv('balancedv2/train_label.csv', index=False, header=False)


In [6]:
# (Hamzah Hafejee, 2022, COMP3611_Coursework_Assessment.ipynb, Comp 3611, University of Leeds)
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

def stratified_split(df, feature):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.25,random_state=42)
    
    # returns 2 sets of indexes for test and train
    # hence, .loc is used on the dataset df to retrieve the corresponding records
    for train_index, test_index in split.split(df,df[feature]):
        strat_train_set = df.loc[train_index]
        strat_test_set = df.loc[test_index]
    
    # for set_ in (strat_train_set, strat_test_set):
    #     set_.drop((feature),axis=1,inplace=True)
        
   
    return strat_train_set, strat_test_set

train_set, test_set = stratified_split(balanced, 'label')

training_set=train_set.drop("label",axis=1)
training_labels=train_set["label"].copy()

testing = test_set.drop("label",axis=1)
testing_labels= test_set["label"].copy()

training_set.to_csv('balanced_data/train_data.csv', index=False, header=False)
training_labels.to_csv('balanced_data/train_label.csv', index=False, header=False)
testing.to_csv('balanced_data/test_data.csv', index=False, header=False)
testing_labels.to_csv('balanced_data/test_label.csv', index=False, header=False)


Manually fixed to make NDBE and neoplasia completely equal